Previous: [Intro](pipeline_intro.ipynb)  ::: Next [pipeline.experiment](pipeline_experiment.ipynb)

# The Big Picture

In [ ]:
%pylab inline
import datajoint as dj
from pipeline import mice, experiment, vis, preprocess, tuning
pylab.rcParams['figure.figsize'] = (8, 8)  # make figures bigger

Below is the outline of processing chain. 

Each schema corresponds to a module in the `pipeline` package *and* the corresponding database on the server.

Each node represents a class in the module *and* a table in the database.

Lines represents dependencies.

* The pipeline starts with the manual information entered in the schemas `mice` and `experiment`.  
* The visual stimulus information is stored in the `vis` schema.  
* The schema `preprocess` extracts traces, infers spikes, and synchronizes two-photon data to the visual stimulus.

In [ ]:
erd = dj.ERD(preprocess.Sync) - 2
erd += dj.ERD(preprocess.Spikes) - 3 
erd += experiment.Session 
erd += dj.ERD(tuning.MonetRF.Map) - 1
erd.draw()

The color code: 
* <span style="background-color: #BBDDBB">Manual</span> (e.g. `experiment.Scan`) is data entered *manually* or by external software.
* <span style="background-color: #BBBBDD">Imported</span> (e.g. `preprocess.Prepare`) is data *imported* from external files by datajoint, requires access to local drives in the lab.
* <span style="background-color: #FFBBBB">Computed</span> (e.g. `preprocess.ComputeTraces`) is *computed* by datajoint from data already in the database without access to external files, can be repopulated in the cloud.
* <span style="background-color: #DDDDDD">Lookup</span> (e.g. `preprocess.Method`) are *lookup* tables containing general information that is not specific to any one experiment.  These often contain processing settings and general  knowledge. 

---
For any table, you can see its definition (attributes, dependencies, comments) by using the `show_definition` method.

In [ ]:
experiment.Session().show_definition();

For any tables or relational expressions derived from them, you may see all the attributes using the `heading` attribute.

In [ ]:
experiment.Session().heading

Note that the attributes above the dividing line `---` comprise the _primary key_, i.e. identifying information used for matching data across different tables.

To preview the contents of a table or of a derived relational expression, simply type it at the IPython prompt.

In [ ]:
experiment.Session()

To fetch the contents of a table or a derived relational expression into a numpy record array, use the fetch method.


In [ ]:
data = experiment.Session().fetch()

Next: [pipeline.experiment](pipeline_experiment.ipynb)